In [1]:
#add main path
import sys
sys.path.append('/Users/lmwgsrwny/Documents/KWS-on-Mamba/')

import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torch_optimizer import Lookahead
from src.data.config import dataset, data_loader, model as model_config, optimizer as optimizer_config, scheduler as scheduler_config, training

# Import custom modules
from src.models.model import KeywordSpottingModel_with_cls
from src.data.data_loader import load_speech_commands_dataset, TFDatasetAdapter, load_bg_noise_dataset
from src.utils.utils import set_memory_GB, print_model_size, log_to_file, plot_learning_curves
from src.utils.augmentations import add_time_shift_and_align, add_silence
from src.utils.train_utils import trainig_loop


/Users/lmwgsrwny/Documents/KWS-on-Mamba/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/lmwgsrwny/Documents/KWS-on-Mamba/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load datasets
train_ds, val_ds, test_ds, silence_ds , info = load_speech_commands_dataset(reduced=True)
# bg_noise_ds = load_bg_noise_dataset()

In [3]:

# Initialize datasets with configurations
pytorch_test_dataset = TFDatasetAdapter(test_ds, None, **dataset, augmentation=[lambda x: add_time_shift_and_align(x)])
#

2025-01-01 12:45:48.786839: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-01-01 12:45:48.973397: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:

# # Create DataLoaders
# train_loader = DataLoader(pytorch_train_dataset, **data_loader, shuffle=True)
# val_loader = DataLoader(pytorch_val_dataset, **data_loader, shuffle=False)


In [5]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print(f"Device: {device}")

Device: mps


In [6]:

# Initialize model
model = KeywordSpottingModel_with_cls(**model_config)

# Loss function
criterion = nn.CrossEntropyLoss().to(device)

# Optimizer
base_optimizer = optim.Adam(model.parameters(), lr=optimizer_config['lr'], weight_decay=optimizer_config['weight_decay'])
optimizer = Lookahead(base_optimizer, **optimizer_config['lookahead'])

# Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, **scheduler_config['reduce_lr_on_plateau'])


In [7]:
# load test data
pytorch_test_dataset = TFDatasetAdapter(test_ds, None, **dataset, augmentation=None)
test_loader = DataLoader(pytorch_test_dataset, **data_loader, shuffle=False)


2025-01-01 12:45:49.757134: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
model.load_state_dict(torch.load('/Users/lmwgsrwny/Documents/KWS-on-Mamba/best_model.pth',map_location=torch.device(device)))


/var/folders/f5/gsx5dxy538z0xyk3rpdwmjqr0000gn/T/ipykernel_32789/965035254.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/Users/lmwgs

<All keys matched successfully>

In [9]:
model = model.to(device)

In [10]:

# Evaluate the model on the test set
accuracy = 0
total = 0

model.eval()

with torch.no_grad():
    for audio, labels in test_loader:
        audio, labels = audio.to(device), labels.to(device)
        outputs = model(audio)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        accuracy += (predicted == labels).sum().item()
test_accuracy = 100 * accuracy / total
print(f'Test Accuracy: {test_accuracy}%')



Test Accuracy: 83.3578792341679%


In [11]:
import pandas as pd
from utils import compute_inference_GPU_mem

# Define missing variables
configs = {'batch_size': 32}  # Example value, adjust as needed
input_dim = model_config['input_dim']  # Example value, adjust as needed
d_model = model_config['d_model']  # Example value, adjust as needed
d_state = model_config['d_state']  # Example value, adjust as needed
d_conv = model_config['d_conv']  # Example value, adjust as needed
expand = model_config['expand']  # Example value, adjust as needed

# Save model size(macs, params) and accuracy
batch_size = configs['batch_size']
macs, params = print_model_size(model, input_size=torch.randn(batch_size, input_dim, d_model-1).to("cuda"))
macs = macs / 1e9
accuracy = test_accuracy
data = {'Model': ['KeywordSpottingModel_RSM_Norm_0-1-2_order_cls_bgnoise'], 'GMACs': [macs], 'Params': [params], 'Accuracy': [accuracy]}
model_config = {'input_dim': input_dim, 'd_model': d_model, 'd_state': d_state, 'd_conv': d_conv, 'expand': expand}
data.update(model_config)
inf_GPU_mem = compute_inference_GPU_mem(model, input=torch.randn(1, input_dim, d_model-1).to("cuda"))
# Inference macs and params
inf_macs, inf_params = print_model_size(model, input_size=torch.randn(1, input_dim, d_model-1).to("cuda"))
inference_data = {'Inference CUDA Mem in MB': [inf_GPU_mem], 'Inference GMACs': [inf_macs / 1e9], 'Inference Params': [inf_params]}
data.update(inference_data)
df = pd.DataFrame(data, index=[0])
df.to_csv('results.csv', mode='a', header=False)

ModuleNotFoundError: No module named 'utils'